# Lenet5 : input : 32 * 32



In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import time
import torch.nn.functional as F
import torch.nn as nn
import matplotlib.pyplot as plt
 
from torchvision import models

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [3]:
# parameters

learning_rate = 0.001
training_epochs = 50
batch_size = 200

In [4]:
#입력 사이즈 맞추기
transforms=transforms.Compose([transforms.Resize((35,35)),transforms.ToTensor()])

mnist_train = dsets.FashionMNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms,
                          download=True)

mnist_test = dsets.FashionMNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms,
                         download=True)

Extracting MNIST_data/FashionMNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/FashionMNIST/raw


Extracting MNIST_data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/FashionMNIST/raw


Extracting MNIST_data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_data/FashionMNIST/raw


Extracting MNIST_data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_data/FashionMNIST/raw
Processing...
Done!


/pytorch/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


In [5]:
# dataset loader
train_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

test_loader = torch.utils.data.DataLoader(dataset=mnist_test,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)


In [6]:
class Lenet5(torch.nn.Module):
  def __init__(self):
    super(Lenet5,self).__init__()

    self.l1=torch.nn.Conv2d(1,6,kernel_size=5,padding=0,stride=1)
    self.x1=torch.nn.Tanh()
    self.l2=torch.nn.AvgPool2d(kernel_size=2,padding=0,stride=2)

    self.l3=torch.nn.Conv2d(6,16,kernel_size=5,padding=0,stride=1)
    self.x2=torch.nn.Tanh()
    self.l4=torch.nn.AvgPool2d(kernel_size=2,padding=0,stride=2)

    self.l5=torch.nn.Flatten()

    self.l6=torch.nn.Linear(16*5*5,120,bias=True)
    self.x3=torch.nn.Tanh()

    self.l7=torch.nn.Linear(120,84,bias=True)
    self.x4=torch.nn.Tanh()

    self.l8=torch.nn.Linear(84,10,bias=True)
  


  def forward(self,x):
    out=self.l1(x)
    out=self.x1(out)
    out=self.l2(out)
    out=self.l3(out)
    out=self.x2(out)
    out=self.l4(out)

    out=out.view(out.size(0),-1)

    out=self.l6(out)
    out=self.x3(out)
    out=self.l7(out)
    out=self.x4(out)
    out=self.l8(out)
    return out

In [7]:
model=Lenet5().to(device)

In [8]:
loss = torch.nn.CrossEntropyLoss().to(device)   
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 
#optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate) 

In [9]:

#train my model
total_batch = len(train_loader)
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in train_loader:
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = loss(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

print('Learning Finished!')

Learning started. It takes sometime.
[Epoch:    1] cost = 0.778190494
[Epoch:    2] cost = 0.497705162
[Epoch:    3] cost = 0.432052433
[Epoch:    4] cost = 0.397401303
[Epoch:    5] cost = 0.371721268
[Epoch:    6] cost = 0.35234648
[Epoch:    7] cost = 0.335923851
[Epoch:    8] cost = 0.321126997
[Epoch:    9] cost = 0.310501486
[Epoch:   10] cost = 0.299784541
[Epoch:   11] cost = 0.290333837
[Epoch:   12] cost = 0.281966954
[Epoch:   13] cost = 0.275036126
[Epoch:   14] cost = 0.265579194
[Epoch:   15] cost = 0.258385926
[Epoch:   16] cost = 0.250548512
[Epoch:   17] cost = 0.247625053
[Epoch:   18] cost = 0.23829484
[Epoch:   19] cost = 0.23162511
[Epoch:   20] cost = 0.226431817
[Epoch:   21] cost = 0.222429737
[Epoch:   22] cost = 0.217887178
[Epoch:   23] cost = 0.20928888
[Epoch:   24] cost = 0.205097631
[Epoch:   25] cost = 0.20114024
[Epoch:   26] cost = 0.19532679
[Epoch:   27] cost = 0.188886195
[Epoch:   28] cost = 0.18424201
[Epoch:   29] cost = 0.181213126
[Epoch:   30]

In [10]:
accuracy = 0
total_batch = len(test_loader)

with torch.no_grad():
  model.eval()

  for X, Y in test_loader:
    Y = Y.to(device)
    X = X.to(device)


    prediction = model(X)
    correct_prediction = torch.argmax(prediction, 1) == Y
    accuracy += correct_prediction.float().mean()

print('Accuracy:', accuracy.item()/total_batch)

Accuracy: 0.8890000915527344
